# WebDAV - request: filename

https://chatgpt.com/c/6824cf43-6738-8005-8733-54b6a77f20ee

In [1]:
from bluer_options import string

from bluer_objects import objects
from bluer_objects.host import signature
from bluer_objects.testing import create_test_asset
from bluer_objects import file, storage, env
from bluer_objects.logger import logger

logger.info(f"{'.'.join(signature())},\nbuilt on {string.pretty_date()}")

🌀  bluer_objects-6.107.1.bluer_ai-12.141.1.bluer_objects-6.107.1.bluer_options-5.97.1.torch-2.2.2.Python 3.12.9.Darwin 23.6.0..Jupyter-Notebook,
built on 14 May 2025, 22:24:07


In [2]:
# filename = "test-00.png"

# filename = "this.yaml"

filename = "subfolder/this.yaml"

In [3]:
object_name = objects.unique_object("test-WebDAV")

local_path=objects.path_of(
        object_name=object_name,
        filename=filename,
    )

🌀  📂 test-WebDAV-2025-05-14-22-24-07-l646v2


In [4]:
assert create_test_asset(object_name)

🌀  bluer_objects.testing.functions.create_test_asset: test-WebDAV-2025-05-14-22-24-07-l646v2
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/test-00.png
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/test-01.png
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/test-02.png
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/test-03.png
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/test-04.png
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/test-05.png
🌀  bluer_objects.file.save.save_image: 512x512x3:uint8 -> /Users/kamangir

---

# create dir

In [5]:
import requests
from requests.auth import HTTPBasicAuth

url = f"{env.WEBDAV_HOSTNAME}/"
folders = [object_name] + [folder for folder in file.path(filename).split("/") if folder]
for folder in folders:
    url = f"{url}{folder}/"

    response = requests.request(
        'MKCOL', 
        url, 
        auth=HTTPBasicAuth(
            env.WEBDAV_LOGIN, 
            env.WEBDAV_PASSWORD,
        )
    )

    if response.status_code in [201, 405]:  # 201 = Created, 405 = Already exists
        logger.info("created {}".format(url.split(env.WEBDAV_HOSTNAME,1)[1]))
    else:
        logger.error(f'failed to create directory: {response.status_code} - {response.text}')
        assert False


🌀  created /test-WebDAV-2025-05-14-22-24-07-l646v2/
🌀  created /test-WebDAV-2025-05-14-22-24-07-l646v2/subfolder/


# upload file

In [6]:
import requests
from requests.auth import HTTPBasicAuth

url = f"{env.WEBDAV_HOSTNAME}/{object_name}/{filename}"

with open(local_path, 'rb') as file_data:
    response = requests.put(
        url, 
        data=file_data, 
        auth=HTTPBasicAuth(
            env.WEBDAV_LOGIN, 
            env.WEBDAV_PASSWORD,
        )
    )

if response.status_code in [200, 201, 204]:
    logger.info(f'uploaded {object_name}/{filename}')
else:
    logger.error(f'failed to upload: {response.status_code} - {response.text}')
    assert False


🌀  uploaded test-WebDAV-2025-05-14-22-24-07-l646v2/subfolder/this.yaml


In [7]:
assert file.delete(local_path)

# download file

In [8]:
import requests
from requests.auth import HTTPBasicAuth

url = f"{env.WEBDAV_HOSTNAME}/{object_name}/{filename}"

response = requests.get(
    url, 
    auth=HTTPBasicAuth(
        env.WEBDAV_LOGIN, 
        env.WEBDAV_PASSWORD
    )
)

if response.status_code == 200:
    with open(local_path, 'wb') as file_:
        file_.write(response.content)
    logger.info(f"downloaded {object_name}/{filename}")
else:
    logger.error(f"failed to download: {response.status_code}")
    assert False


🌀  downloaded test-WebDAV-2025-05-14-22-24-07-l646v2/subfolder/this.yaml


In [9]:
! ls {local_path}

/Users/kamangir/storage/abcli/test-WebDAV-2025-05-14-22-24-07-l646v2/subfolder/this.yaml


---

In [10]:
# END